# Shapenet Rendering
- Blender script source: https://github.com/panmari/stanford-shapenet-renderer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import glob
import cv2
import imageio
import shutil

import subprocess
import numpy as np

from modules.ShapeNetHandler import ShapeNetHandler
import modules.utils

# Load ShapeNet Handler

In [3]:
shapenet_root = "/hdd/mliuzzolino/ShapeNet/data/ShapeNetCore.v2"
shapenet_handler = ShapeNetHandler(shapenet_root)

## Print available categories by name
- Can use name to condition random sampling

In [4]:
# shapenet_handler.print_categories()

## Randomly Sample a filepath

In [5]:
obj_filepath, synset_id, instance_id = shapenet_handler.sample_obj(category_name='car')

SynsetID: 02958343 -- InstanceID: 335b3ad6f7a96bf251a95aaa6caba1d3 -- Name: car,auto,automobile,machine,motorcar


## Background Images

In [6]:
background_imgs_root = '/hdd/mliuzzolino/Places2/places365_standard/train'
background_img_paths = glob.glob(f'{background_imgs_root}/*/*')

## Output Path

In [8]:
_OUTPUT_ROOT = 'render_output'

# Render

In [9]:
_N_VIEW_ANGLES = 72
_N_MODELS_TO_RENDER = 20
_OVERWRITE = True
_RENDER_SCRIPT = "scripts/blender_render.py"
_TARGET_CATEGORY = ''

## Run Blender

In [ ]:
subprocess_outputs = []
try:
    for img_i in range(_N_MODELS_TO_RENDER):
        obj_filepath, synset_id, instance_id = shapenet_handler.sample_obj(category_name=_TARGET_CATEGORY)
        _OUTPUT_PATH = os.path.join(_OUTPUT_ROOT, f'synsetID_{synset_id}', instance_id)

        if not os.path.exists(_OUTPUT_PATH) or _OVERWRITE:
            if _OVERWRITE and os.path.exists(_OUTPUT_PATH):
                shutil.rmtree(_OUTPUT_PATH)
            cmd = f"blender --background --python {_RENDER_SCRIPT} -- "
            cmd += f"--output_folder {_OUTPUT_PATH} --views {_N_VIEW_ANGLES} "
            cmd += f"{obj_filepath}"
            out = subprocess.check_output(cmd.split(" "), shell=False)
            subprocess_outputs.append(out)
        else:
            print("Skipping.")
except KeyboardInterrupt:
    print("\nEnding early.")

## Load Images and Animate

In [10]:
def add_background(imgs, background_img):
    RGB_img = imgs[0]
    mask = imgs[1]
    RGB_img[np.where(~mask)] = background_img[np.where(~mask)]

    return RGB_img, mask

In [14]:
_LOAD_LIMIT = 20

limit_reached = False

all_imgs = []
rendered_synset_ids = glob.glob(f"{_OUTPUT_ROOT}/*")
np.random.shuffle(rendered_synset_ids)
for rendered_synset_id in rendered_synset_ids:
    rendered_instance_ids = glob.glob(f"{rendered_synset_id}/*")
    for rendered_instance_id in rendered_instance_ids:
        sys.stdout.write(f"\rProcessed model {len(all_imgs)+1}/{_LOAD_LIMIT}...")
        sys.stdout.flush()
        rendered_img_paths = np.sort(glob.glob(f"{rendered_instance_id}/*"))
        
        rendered_imgs = [modules.utils.read_image(img_path, same_size=True) 
                             for img_path in rendered_img_paths]
        
        random_background_path = np.random.choice(background_img_paths)
        background_img = imageio.imread(random_background_path)
        shape = rendered_imgs[0][0].shape[:2][::-1]
        background_img = cv2.resize(background_img, shape)
        rendered_imgs = [add_background(img, background_img) for img in rendered_imgs]
        if len(rendered_imgs) != _N_VIEW_ANGLES:
            print(f"\t**Error: Insufficient angles! Actual: {len(rendered_imgs)} - Expected: {_N_VIEW_ANGLES}")
            continue
            
        combined_imgs = modules.utils.combine_imgs(rendered_imgs)
        all_imgs.append(combined_imgs)
        
        if len(all_imgs) >= _LOAD_LIMIT:
            print("Breaking..")
            limit_reached = True
            break
    
    if limit_reached:
        break

Processed model 1/20...
Processed model 2/20...
Processed model 3/20...
Processed model 4/20...
Processed model 5/20...
Processed model 6/20...
Processed model 7/20...
Processed model 8/20...
Processed model 9/20...
Processed model 10/20...
Processed model 11/20...
Processed model 12/20...
Processed model 13/20...
Processed model 14/20...
Processed model 15/20...
Processed model 16/20...
Processed model 17/20...
Processed model 18/20...
Processed model 19/20...
Processed model 20/20...
Breaking..


In [ ]:
# Stack all images
stacked_imgs = modules.utils.stack_all_imgs(all_imgs, nrow=2)

In [ ]:
modules.utils.toAnimation(stacked_imgs, figsize=(12,12), interval=100, savepath='test.mp4', fps=15)